In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.1. Data Imputation')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.1. Data Imputation/Output')
cur_date = "080823"

library(missForest)
library(imputeLCMD)
library(readxl)
library(openxlsx)
library(tidyverse)

# reading in file
well_df = data.frame(read_excel("Input/Union_WellData_080723.xlsx", sheet = 2))

# converting some cols to numeric
numeric_df = data.frame(apply(well_df[,c(5,7,9,11,13,14,19)], 2, as.numeric))
# converting one col to charcter
well_df$Landuse = as.character(well_df$Landuse)
recombined_df = cbind(well_df[,-c(5,7,9,11,13,14,19)], numeric_df)
well_df = recombined_df[,c(1:4,13,5,14,6,15,9:11,17:19,12,7,16,8)]

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
and has been superseded by the norm2 package.

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownam

Warning message:
“Expecting numeric in R2186 / R2186C18: got 'NA'”
Warning message:
“Expecting numeric in Q2187 / R2187C17: got 'NA'”
Warning message:
“Expecting numeric in R2187 / R2187C18: got 'NA'”
Warning message:
“Expecting numeric in Q2188 / R2188C17: got 'NA'”
Warning message:
“Expecting numeric in R2188 / R2188C18: got 'NA'”
Warning message:
“Expecting numeric in Q2189 / R2189C17: got 'NA'”
Warning message:
“Expecting numeric in R2189 / R2189C18: got 'NA'”
Warning message:
“Expecting numeric in Q2190 / R2190C17: got 'NA'”
Warning message:
“Expecting numeric in R2190 / R2190C18: got 'NA'”
Warning message:
“Expecting numeric in Q2191 / R2191C17: got 'NA'”
Warning message:
“Expecting numeric in R2191 / R2191C18: got 'NA'”
Warning message:
“Expecting numeric in Q2192 / R2192C17: got 'NA'”
Warning message:
“Expecting numeric in R2192 / R2192C18: got 'NA'”
Warning message:
“Expecting numeric in Q2193 / R2193C17: got 'NA'”
Warning message:
“Expecting numeric in R2193 / R2193C18: got '

Warning message:
“Expecting numeric in Q2248 / R2248C17: got 'NA'”
Warning message:
“Expecting numeric in R2248 / R2248C18: got 'NA'”
Warning message:
“Expecting numeric in Q2249 / R2249C17: got 'NA'”
Warning message:
“Expecting numeric in R2249 / R2249C18: got 'NA'”
Warning message:
“Expecting numeric in Q2250 / R2250C17: got 'NA'”
Warning message:
“Expecting numeric in R2250 / R2250C18: got 'NA'”
Warning message:
“Expecting numeric in Q2251 / R2251C17: got 'NA'”
Warning message:
“Expecting numeric in R2251 / R2251C18: got 'NA'”
Warning message:
“Expecting numeric in Q2252 / R2252C17: got 'NA'”
Warning message:
“Expecting numeric in R2252 / R2252C18: got 'NA'”
Warning message:
“Expecting numeric in Q2253 / R2253C17: got 'NA'”
Warning message:
“Expecting numeric in R2253 / R2253C18: got 'NA'”
Warning message:
“Expecting numeric in Q2254 / R2254C17: got 'NA'”
Warning message:
“Expecting numeric in R2254 / R2254C18: got 'NA'”
Warning message:
“Expecting numeric in Q2255 / R2255C17: got '

Warning message:
“Expecting numeric in R2309 / R2309C18: got 'NA'”
Warning message:
“Expecting numeric in Q2310 / R2310C17: got 'NA'”
Warning message:
“Expecting numeric in R2310 / R2310C18: got 'NA'”
Warning message:
“Expecting numeric in Q2311 / R2311C17: got 'NA'”
Warning message:
“Expecting numeric in R2311 / R2311C18: got 'NA'”
Warning message:
“Expecting numeric in Q2312 / R2312C17: got 'NA'”
Warning message:
“Expecting numeric in R2312 / R2312C18: got 'NA'”
Warning message:
“Expecting numeric in Q2313 / R2313C17: got 'NA'”
Warning message:
“Expecting numeric in R2313 / R2313C18: got 'NA'”
Warning message:
“Expecting numeric in Q2314 / R2314C17: got 'NA'”
Warning message:
“Expecting numeric in R2314 / R2314C18: got 'NA'”
Warning message:
“Expecting numeric in Q2315 / R2315C17: got 'NA'”
Warning message:
“Expecting numeric in R2315 / R2315C18: got 'NA'”
Warning message:
“Expecting numeric in Q2316 / R2316C17: got 'NA'”
Warning message:
“Expecting numeric in R2316 / R2316C18: got '

Warning message:
“Expecting numeric in Q2371 / R2371C17: got 'NA'”
Warning message:
“Expecting numeric in R2371 / R2371C18: got 'NA'”
Warning message:
“Expecting numeric in Q2372 / R2372C17: got 'NA'”
Warning message:
“Expecting numeric in R2372 / R2372C18: got 'NA'”
Warning message:
“Expecting numeric in Q2373 / R2373C17: got 'NA'”
Warning message:
“Expecting numeric in R2373 / R2373C18: got 'NA'”
Warning message:
“Expecting numeric in Q2374 / R2374C17: got 'NA'”
Warning message:
“Expecting numeric in R2374 / R2374C18: got 'NA'”
Warning message:
“Expecting numeric in Q2375 / R2375C17: got 'NA'”
Warning message:
“Expecting numeric in R2375 / R2375C18: got 'NA'”
Warning message:
“Expecting numeric in Q2376 / R2376C17: got 'NA'”
Warning message:
“Expecting numeric in R2376 / R2376C18: got 'NA'”
Warning message:
“Expecting numeric in Q2377 / R2377C17: got 'NA'”
Warning message:
“Expecting numeric in R2377 / R2377C18: got 'NA'”
Warning message:
“Expecting numeric in Q2378 / R2378C17: got '

...

Warning message:
“Expecting numeric in R2801 / R2801C18: got 'NA'”
Warning message:
“Expecting numeric in Q2802 / R2802C17: got 'NA'”
Warning message:
“Expecting numeric in R2802 / R2802C18: got 'NA'”
Warning message:
“Expecting numeric in Q2803 / R2803C17: got 'NA'”
Warning message:
“Expecting numeric in R2803 / R2803C18: got 'NA'”
Warning message:
“Expecting numeric in Q2804 / R2804C17: got 'NA'”
Warning message:
“Expecting numeric in R2804 / R2804C18: got 'NA'”
Warning message:
“Expecting numeric in Q2805 / R2805C17: got 'NA'”
Warning message:
“Expecting numeric in R2805 / R2805C18: got 'NA'”
Warning message:
“Expecting numeric in Q2806 / R2806C17: got 'NA'”
Warning message:
“Expecting numeric in R2806 / R2806C18: got 'NA'”
Warning message:
“Expecting numeric in Q2807 / R2807C17: got 'NA'”
Warning message:
“Expecting numeric in R2807 / R2807C18: got 'NA'”
Warning message:
“Expecting numeric in Q2808 / R2808C17: got 'NA'”
Warning message:
“Expecting numeric in R2808 / R2808C18: got '

Warning message:
“Expecting numeric in Q2863 / R2863C17: got 'NA'”
Warning message:
“Expecting numeric in R2863 / R2863C18: got 'NA'”
Warning message:
“Expecting numeric in Q2864 / R2864C17: got 'NA'”
Warning message:
“Expecting numeric in R2864 / R2864C18: got 'NA'”
Warning message:
“Expecting numeric in Q2865 / R2865C17: got 'NA'”
Warning message:
“Expecting numeric in R2865 / R2865C18: got 'NA'”
Warning message:
“Expecting numeric in Q2866 / R2866C17: got 'NA'”
Warning message:
“Expecting numeric in R2866 / R2866C18: got 'NA'”
Warning message:
“Expecting numeric in Q2867 / R2867C17: got 'NA'”
Warning message:
“Expecting numeric in R2867 / R2867C18: got 'NA'”
Warning message:
“Expecting numeric in Q2868 / R2868C17: got 'NA'”
Warning message:
“Expecting numeric in R2868 / R2868C18: got 'NA'”
Warning message:
“Expecting numeric in Q2869 / R2869C17: got 'NA'”
Warning message:
“Expecting numeric in R2869 / R2869C18: got 'NA'”
Warning message:
“Expecting numeric in Q2870 / R2870C17: got '

Warning message:
“Expecting numeric in R2924 / R2924C18: got 'NA'”
Warning message:
“Expecting numeric in Q2925 / R2925C17: got 'NA'”
Warning message:
“Expecting numeric in R2925 / R2925C18: got 'NA'”
Warning message:
“Expecting numeric in Q2926 / R2926C17: got 'NA'”
Warning message:
“Expecting numeric in R2926 / R2926C18: got 'NA'”
Warning message:
“Expecting numeric in Q2927 / R2927C17: got 'NA'”
Warning message:
“Expecting numeric in R2927 / R2927C18: got 'NA'”
Warning message:
“Expecting numeric in Q2928 / R2928C17: got 'NA'”
Warning message:
“Expecting numeric in R2928 / R2928C18: got 'NA'”
Warning message:
“Expecting numeric in Q2929 / R2929C17: got 'NA'”
Warning message:
“Expecting numeric in R2929 / R2929C18: got 'NA'”
Warning message:
“Expecting numeric in Q2930 / R2930C17: got 'NA'”
Warning message:
“Expecting numeric in R2930 / R2930C18: got 'NA'”
Warning message:
“Expecting numeric in Q2931 / R2931C17: got 'NA'”
Warning message:
“Expecting numeric in R2931 / R2931C18: got '

Warning message:
“Expecting numeric in Q2986 / R2986C17: got 'NA'”
Warning message:
“Expecting numeric in R2986 / R2986C18: got 'NA'”
Warning message:
“Expecting numeric in Q2987 / R2987C17: got 'NA'”
Warning message:
“Expecting numeric in R2987 / R2987C18: got 'NA'”
Warning message:
“Expecting numeric in Q2988 / R2988C17: got 'NA'”
Warning message:
“Expecting numeric in R2988 / R2988C18: got 'NA'”
Warning message:
“Expecting numeric in Q2989 / R2989C17: got 'NA'”
Warning message:
“Expecting numeric in R2989 / R2989C18: got 'NA'”
Warning message:
“Expecting numeric in Q2990 / R2990C17: got 'NA'”
Warning message:
“Expecting numeric in R2990 / R2990C18: got 'NA'”
Warning message:
“Expecting numeric in Q2991 / R2991C17: got 'NA'”
Warning message:
“Expecting numeric in R2991 / R2991C18: got 'NA'”
Warning message:
“Expecting numeric in Q2992 / R2992C17: got 'NA'”
Warning message:
“Expecting numeric in R2992 / R2992C18: got 'NA'”
Warning message:
“Expecting numeric in Q2993 / R2993C17: got '

Warning message:
“Expecting numeric in R3047 / R3047C18: got 'NA'”
Warning message:
“Expecting numeric in Q3048 / R3048C17: got 'NA'”
Warning message:
“Expecting numeric in R3048 / R3048C18: got 'NA'”
Warning message:
“Expecting numeric in Q3049 / R3049C17: got 'NA'”
Warning message:
“Expecting numeric in R3049 / R3049C18: got 'NA'”
Warning message:
“Expecting numeric in Q3050 / R3050C17: got 'NA'”
Warning message:
“Expecting numeric in R3050 / R3050C18: got 'NA'”
Warning message:
“Expecting numeric in Q3051 / R3051C17: got 'NA'”
Warning message:
“Expecting numeric in R3051 / R3051C18: got 'NA'”
Warning message:
“Expecting numeric in Q3052 / R3052C17: got 'NA'”
Warning message:
“Expecting numeric in R3052 / R3052C18: got 'NA'”
Warning message:
“Expecting numeric in Q3053 / R3053C17: got 'NA'”
Warning message:
“Expecting numeric in R3053 / R3053C18: got 'NA'”
Warning message:
“Expecting numeric in Q3054 / R3054C17: got 'NA'”
Warning message:
“Expecting numeric in R3054 / R3054C18: got '

In [2]:
head(well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,NA,ND
2,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Mn,440,D
3,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Cr,NA,ND
4,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,NA,ND
5,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Mn,690,D
6,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Cr,NA,ND


Imputing missing data. Random forest will be used to input casing depth, well depth, static water depth, flow rate, and pH. Quantile regression imputation of left-censored data (QRILC) will be used to impute each metal concentration.

Typically background filters would be implemented for all data, but only the variable `Concentration` is missing a significant portion of data.

First, we're only keeping data that has a latitude and longtitude, since quite a few of the variables have a significant amount of missing data.

In [3]:
smaller_well_df = well_df[!is.na(well_df$Longtitude),]
head(smaller_well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,NA,ND
2,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Mn,440,D
3,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Cr,NA,ND
4,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,NA,ND
5,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Mn,690,D
6,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Cr,NA,ND


In [4]:
# original number of records
dim(well_df)

# records kept
dim(smaller_well_df)

[1] 3069   19

[1] 2145   19

In [5]:
# determining if other variables need to be imputed
dim(smaller_well_df[!is.na(smaller_well_df$Geology),])
dim(smaller_well_df[!is.na(smaller_well_df$Soil_Type),])
dim(smaller_well_df[!is.na(smaller_well_df$Landuse),])
dim(smaller_well_df[!is.na(smaller_well_df$Stream_Distance),])
dim(smaller_well_df[!is.na(smaller_well_df$Elevation),])

[1] 2145   19

[1] 2145   19

[1] 1641   19

[1] 2145   19

[1] 1641   19

I would impute `Landuse`, but not sure that should be converted into a factor, but `Elevation` will be imputed.

In [6]:
# splitting the dataset based on the metal
preimputed_df = smaller_well_df %>%
    group_by(Metal) %>%
    group_split()

arsenic_df = preimputed_df[[1]]
chromium_df = preimputed_df[[2]]
manganese_df = preimputed_df[[3]]

# RF Imputation

In [7]:
# imputing cols 5-9 and 16 using RF
random_forest_imputation = function(dataset){
    
    imputed_RF_object = missForest(as.matrix(dataset[,c(5:9,16)]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:4], imputed_RF_df[,1:5], dataset[,10:15], imputed_RF_df[,6],
                                       dataset[,17:19])) 
    
    #idk why i have to do this
    colnames(imputed_final_df)[16] = c("Elevation")
    
    return(imputed_final_df)
}

# calling fn
rf_imputed_arsenic_df = random_forest_imputation(arsenic_df)
rf_imputed_chromium_df = random_forest_imputation(chromium_df)
rf_imputed_manganese_df = random_forest_imputation(manganese_df)

head(rf_imputed_arsenic_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,NA,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,NA,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Carolina Slate Belt - CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,Ar,NA,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Carolina Slate Belt - CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,Ar,35,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Carolina Slate Belt - CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,Ar,NA,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Carolina Slate Belt - CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,Ar,53,D


# QRILC Imputation

In [8]:
# QRILC
QRILC_imputation = function(dataset){
    # """
    # Creating a function to generate missing data
    # :param (input): df with missing data
    # :output: 1 imputed df
    # """
  
    # normalizing data since that's what the QRILC function wants
    # ended up imputing another col (health dept id), since this function needs > 1 col to impute
    QRILC_prep = dataset[,c(2,18)] %>%
         mutate_all(., function(x) log2(x)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log2_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log2_df %>%
        mutate_all(., function(x) 2^x)
    
    # adding back in other data
    QRILC_df = data.frame(cbind(dataset[,1:17], QRILC_df[,2], dataset[,19]))
    colnames(QRILC_df)[18:19] = c("Concentration", "Detect_Concentration")


   return(QRILC_df)
}

# calling fn
imputed_arsenic_df = QRILC_imputation(rf_imputed_arsenic_df)
imputed_chromium_df = QRILC_imputation(rf_imputed_chromium_df)
imputed_manganese_df = QRILC_imputation(rf_imputed_manganese_df)

In [9]:
# combining in 1 df
imputed_df = rbind(imputed_arsenic_df, imputed_chromium_df, imputed_manganese_df)
head(imputed_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,1.531414,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,2.348528,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Carolina Slate Belt - CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,Ar,2.043848,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Carolina Slate Belt - CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,Ar,35.000000,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Carolina Slate Belt - CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,Ar,2.297118,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Carolina Slate Belt - CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,Ar,53.000000,D


In [10]:
# exporting results
write.xlsx(imputed_df, paste0(Output,"/", "Imputed_Well_Data_", cur_date, ".xlsx"), rowNames = FALSE)